In [1]:
import pandas as pd
import ast
import re

In [6]:
def get_list_columns(dataframe):
    big_list = []
    for row in dataframe.iterrows():
        x = row[1][6]
        l = []
        for s in x.split(", "):
            star = re.sub(r"[\[\]]",'',s).strip()
            l.append(str(star[1:-1]))
        big_list.append(l)
    new_df = pd.DataFrame({'investor_list' :big_list})
    dataframe['investor'] = new_df['investor_list']
    return dataframe

In [7]:
# Reading data, cleaning column names
df = pd.read_csv('unicorn2021.csv')
df.rename(columns={'Date Joined': 'date',
                   'Select Investors': 'investors',
                   'Valuation ($B)': 'BValuation'},
          inplace=True)
df['date'] = pd.to_datetime(df.date)
df['year'] = pd.DatetimeIndex(df.date).year
df.dropna(inplace=True)
df.head()

,Company,BValuation,date,Country,City,Industry,investors,Level,year
0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial Intelligence,"['Sequoia Capital China', 'SIG Asia Investment...",hectocorn,2017
1,SpaceX,100.3,2012-12-01,United States,Hawthorne,Other,"['Founders Fund', 'Draper Fisher Jurvetson', '...",hectocorn,2012
2,Stripe,95.0,2014-01-23,United States,San Francisco,Fintech,"['Khosla Ventures', 'LowercaseCapital', 'capit...",decacorn,2014
3,Klarna,45.6,2011-12-12,Sweden,Stockholm,Fintech,"['Institutional Venture Partners', 'Sequoia Ca...",decacorn,2011
4,Canva,40.0,2018-01-08,Australia,Surry Hills,Internet software & services,"['Sequoia Capital China', 'Blackbird Ventures'...",decacorn,2018


In [9]:
# Cleaning Investors column
df = df[df.investors.str[:1] == '['].reset_index()
df.drop('index', axis = 1, inplace = True)

df = get_list_columns(df)
df.drop('investors', axis = 1, inplace = True)
df = df.explode('investor').reset_index()

In [12]:
df.drop_duplicates(inplace = True)
df

,index,Company,BValuation,date,Country,City,Industry,Level,year,investor
0,0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial Intelligence,hectocorn,2017,Sequoia Capital China
1,0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial Intelligence,hectocorn,2017,SIG Asia Investments
2,0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial Intelligence,hectocorn,2017,Sina Weibo
3,0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial Intelligence,hectocorn,2017,Softbank Group
4,1,SpaceX,100.3,2012-12-01,United States,Hawthorne,Other,hectocorn,2012,Founders Fund
...,...,...,...,...,...,...,...,...,...,...
2598,918,ReliaQuest,1.0,2021-12-01,United States,Tampa,Cybersecurity,unicorn,2021,FTV Capital
2599,918,ReliaQuest,1.0,2021-12-01,United States,Tampa,Cybersecurity,unicorn,2021,Ten Eleven Ventures
2600,919,Pet Circle,1.0,2021-12-07,Australia,Alexandria,E-commerce & direct-to-consumer,unicorn,2021,Prysm Capital
2601,919,Pet Circle,1.0,2021-12-07,Australia,Alexandria,E-commerce & direct-to-consumer,unicorn,2021,Baillie Gifford & Co.


In [13]:
df.to_csv('unicorn-clean.csv')